In [230]:
import mesa
from mesa.experimental import JupyterViz
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import seaborn as sns

In [231]:
def followers(model):
        dancers = sum(1 for bee in model.schedule.agents if isinstance(bee, Bee) and bee.following_dance)

        percentage = ( dancers  / model.num_agents)*100

        return percentage
    
def average_food_quality(model):
    # Calculating the average quality of food sources in the model
    total_food_qual_sum = sum(food.quality for food in model.schedule.agents if isinstance(food, Food))
    num_food_sources = sum(1 for food in model.schedule.agents if isinstance(food, Food))

    average_food_quality = total_food_qual_sum / num_food_sources if num_food_sources > 0 else 0

    return average_food_quality
    
    
def average_memory(model):
    # Calculating the average memory quality for each bee and then the overall average
    total_quality_sum = sum(sum(bee.memory.values()) / len(bee.memory) if bee.memory else 0 
                            for bee in model.schedule.agents if isinstance(bee, Bee))
    num_bees = sum(1 for bee in model.schedule.agents if isinstance(bee, Bee))

    average_memory_quality = total_quality_sum / num_bees if num_bees > 0 else 0

    return average_memory_quality


measures = {
    "average_food_quality": average_food_quality,
    "average_memory": average_memory
}

def constant():
    return 1

    

In [232]:
class Hive(mesa.Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos


class Food(mesa.Agent):
    def __init__(self, unique_id, model, quality):
        super().__init__(unique_id, model)
        self.quality = quality
        


In [233]:
class Bee(mesa.Agent):
    def __init__(self, unique_id, model, quality_threshold, experiences, no_memories, memory_test, mode_test):
        super().__init__(unique_id, model)
        self.memory = {}
        self.experiences = experiences
        self.no_memories = no_memories
        self.recent_steps = deque(maxlen=3)
        self.target_food_location = None
        self.in_hive = False
        self.move_mode = False
        self.dancing_baby = False 
        self.following_dance = False
        self.memory_test = memory_test 
        self.mode_test = mode_test
        self.quality_threshold = quality_threshold
      
    
       
    def decision(self):
        prob = self.evaluation()  
        return np.random.random() > prob
    

    def evaluation(self):
        
        weight_memory = 0.35
        weight_experience = 0.35
        weight_quality_difference = 0.20
        weight_distance = 0.1
        
        
        if self.mode_test == True:
            
            max_memory_value = 100 
            memory_factor_quality = self.memory_test / max_memory_value 
            memory_factor = 1
            memory_factor_total = memory_factor_quality * memory_factor
            
            max_experience_value = 100
            experience_factor = self.experiences / max_experience_value
                        
            max_quality = 100
            quality_factor = max(0, self.model.quality_food_waggle - self.quality_threshold) / max_quality
            
            distance_term = 0.6
        
        
        else:
             
            max_memory_value = 100 
            memory_factor_quality = (sum(self.memory.values()) / len(self.memory)) / max_memory_value if self.memory else 0
            memory_factor = 1
            memory_factor_total = memory_factor_quality * memory_factor

            max_experience_value = 100
            experience_factor = self.experiences / max_experience_value

            max_distance = 100
            distance_factor = self.model.last_food_distance / max_distance 
            distance_term = weight_distance / (distance_factor + 0.001)
            distance_term = min(distance_term, weight_distance)
            
            max_quality = 100
            quality_factor = max(0, self.model.last_food_quality - self.quality_threshold) / max_quality
            
    
        prob = (weight_memory * memory_factor_total) + \
               (weight_experience * experience_factor) + \
               (weight_quality_difference * quality_factor) + \
               distance_term

        prob = max(0, min(prob, 1))

        return prob

    

    def move_gradually_towards(self, target_pos):
        current_x, current_y = self.pos
        target_x, target_y = target_pos

        # Calcular dirección hacia la posición objetivo
        direction_x = np.sign(target_x - current_x)
        direction_y = np.sign(target_y - current_y)

        # Añadir ruido aleatorio
        noise_x, noise_y = np.random.randint(-1, 2), np.random.randint(-1, 2)

        # Nueva posición teniendo en cuenta el ruido
        new_x = max(0, min(self.model.width - 1, current_x + direction_x + noise_x))
        new_y = max(0, min(self.model.height - 1, current_y + direction_y + noise_y))

        # Mover agente
        self.model.grid.move_agent(self, (new_x, new_y))


  
    def waggle_dance(self):
            for bee in self.model.schedule.agents:
                if isinstance(bee, Bee) and not bee.dancing_baby:
                    bee.receive_waggle_dance(self.model.last_food_direction, self.model.last_food_distance, self.model.last_food_quality)

    
    def receive_waggle_dance(self, direction, distance, quality):
            target_x, target_y = self.calculate_target_position_followers(direction, distance)
            self.target_food_location = (int(target_x), int(target_y))

    
    
    def calculate_target_position_followers(self, direction, distance):
            # Normalizar la dirección para obtener un vector unitario
            dx, dy = direction
            length = np.sqrt(dx**2 + dy**2)
            if length != 0:
                dx, dy = dx / length, dy / length

            # Escalar el vector unitario por la distancia
            target_dx = round(dx * distance)
            target_dy = round(dy * distance)

            # Calcular la nueva posición objetivo basada en la dirección y distancia
            target_x = self.pos[0] + target_dx
            target_y = self.pos[1] + target_dy
            
            target_x += np.random.normal(0,self.model.noise)
            target_y += np.random.normal(0,self.model.noise)

            
            return target_x, target_y


    
    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        # 
        possible_steps = [step for step in possible_steps 
                          if self.is_within_bounds(step) and step not in self.recent_steps]
            
            
        new_position = self.random.choice(possible_steps)
        self.recent_steps.appendleft(new_position)
        
        self.model.grid.move_agent(self, new_position)
        

    def is_within_bounds(self, pos):
         x, y = pos
         # Ajustar los límites para mantener las abejas dentro de un margen del borde
         return 1 <= x < self.model.width - 1 and 1 <= y < self.model.height - 1
        
    
    
    def found_food(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        food = next((agent for agent in cellmates if isinstance(agent, Food)), None)

        if food:
            if self.mode_test:
                a = 1
            else: 
                a = 1.5
                
            if food.quality >= a*self.quality_threshold:
                self.dancing_baby = True
                self.model.return_to_hive = True
                self.model.last_food_quality = food.quality
                self.model.last_food_direction, self.model.last_food_distance = self.calculate_food_info(self.pos)

            #if self.experiences != 0:
             #       self.experiences += 10
                    
            if self.no_memories is not True:
                if self.pos in self.memory:
                    self.memory[self.pos] = food.quality

                elif len(self.memory) >= 4 and self.pos not in self.memory.keys():
                   # Eliminar la entrada más antigua si la memoria está llena
                    oldest_entry = next(iter(self.memory))
                    del self.memory[oldest_entry]

                elif len(self.memory) < 4 and self.pos not in self.memory.keys():
                        self.memory[self.pos] = food.quality
            
            

          
    
    def calculate_food_info(self, food_pos):
        # Calcula la dirección y distancia desde la posición actual hasta la comida
        direction = self.get_direction_to(food_pos)
        distance = self.get_distance_to(food_pos)
        
        return direction, distance
    

    def get_direction_to(self, food_pos):
        # Diferencia de coordenadas entre la posición actual y el objetivo
        dx = food_pos[0] - 100 //2
        dy = food_pos[1] - 100//2
        
        dx += np.random.normal(0, self.model.noise) 
        dy += np.random.normal(0, self.model.noise) 
        
        return dx, dy
    

    def get_distance_to(self, food_pos):
        # Pitagoras
        distance = np.sqrt((food_pos[0] - 100//2) ** 2 + (food_pos[1] - 100//2) ** 2)
        distance += np.random.normal(0, self.model.noise)
    
        return distance
    

    def step(self):
        hive_pos = (self.model.width // 2, self.model.height // 2)
        if self.model.return_to_hive:
            self.move_gradually_towards(hive_pos)
            if self.pos == hive_pos:
                self.in_hive = True
                
                
        elif self.in_hive:
            if self.dancing_baby:
                self.pos = hive_pos
            elif self.target_food_location:
                if self.decision():
                    self.following_dance = True
                    self.move_gradually_towards(self.target_food_location)
                    if self.pos == self.target_food_location:
                        self.target_food_location = None
                        self.in_hive = False
                        self.move_mode = True 
                else:
                    self.in_hive = False
                    self.move_mode = True 
                
        elif self.move_mode:
            self.move()
                    
        else:
            self.move()
            self.found_food()


In [234]:
# WORLD
class BeeForagingModel(mesa.Model):

    def __init__(self, N, num_food, width, height, noise, quality_threshold, av_food_qual, av_experiences, quality_food_waggle,
                 no_memories, mode_test, memory_test):
        super().__init__()
        self.num_agents = N
        self.num_food = num_food
        self.noise = noise
        self.running = True
        self.width = width
        self.height = height
        self.return_to_hive = False 
        self.quality_threshold = quality_threshold
        
        self.last_food_direction = None
        self.last_food_distance = None
        self.last_food_quality = None
        
        self.num_dancers = 0
        self.av_food_qual = av_food_qual if av_food_qual else 0
        self.av_experiences = av_experiences if av_experiences else 1
        self.no_memories = no_memories if no_memories else False
        self.quality_food_waggle = quality_food_waggle if quality_food_waggle else 0
        
        self.memory_test = memory_test if memory_test else 0
        self.mode_test = mode_test if mode_test else False
        
        
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)
        
         
        self.datacollector = mesa.DataCollector(model_reporters={"followers": followers})
        
        
        if self.mode_test == True:
            
            if self.av_experiences == 0:
                    self.no_memories = True
                     
            # Create agents
            for i in range(self.num_agents):  
                bee = Bee(i, self, quality_threshold = self.quality_threshold, experiences = self.av_experiences ,  no_memories = self.no_memories,
                       memory_test = self.memory_test, mode_test = self.mode_test )
                self.schedule.add(bee)
                self.grid.place_agent(bee, (width//2, height//2))
                    
             # Create food
            for i in range(self.num_food):
                x, y = self.place_near_edges(self.width, self.width)


                food = Food(i + N, self, quality=self.av_food_qual)
                self.grid.place_agent(food, (x, y))

            
        else:
        
            # Create agents
            for i in range(self.num_agents):

                if self.av_experiences == 0:
                    experience = 0
                else:
                    experience = max(1, min(100, int(np.random.normal(self.av_experiences, 1))))
                
                b = max(1, min(100, int(np.random.normal(self.quality_threshold, 10))))
                
                bee = Bee(i, self , quality_threshold = b, experiences = experience , no_memories = self.no_memories, 
                           memory_test = self.memory_test,  mode_test = self.mode_test )
                self.schedule.add(bee)
                self.grid.place_agent(bee, (width//2, height//2))

            for i in range(self.num_food):
                x, y = self.place_near_edges(self.width, self.width)

                a = max(1, min(100, int(np.random.normal(self.av_food_qual, 1))))
                food = Food(i + N, self, quality=a)
                self.grid.place_agent(food, (x, y))   


        
    def place_near_edges(self, width, height):
            # Colocar comida cerca de los bordes
            if np.random.random() < 0.7:
                # Cerca de los bordes verticales
                x = np.random.choice([np.random.randint(1, width // 4), np.random.randint(3 * width // 4, width - 1)])
            else:
                # Cerca de los bordes horizontales
                x = np.random.randint(1, width)

            if np.random.random() < 0.7:
                y = np.random.choice([np.random.randint(1, height // 4), np.random.randint(3 * height // 4, height - 1)])
            else:
                y = np.random.randint(1, height)

            return x, y



    def all_bees_in_hive(self):
            for agent in self.schedule.agents:
                if isinstance(agent, Bee) and not agent.in_hive:
                    return False
            return True


    def step(self):
        self.schedule.step()
        if self.all_bees_in_hive() and self.return_to_hive:
            for bee in self.schedule.agents:
                if isinstance(bee, Bee) and bee.dancing_baby:
                    bee.waggle_dance()
                    break  # Solo la abeja que realiza el waggle dance lo hace
            self.return_to_hive = False  # Permitir que las abejas reanuden el forrajeo
        self.datacollector.collect(self)

         # Create hive
        hive = Hive("HIVE", self, (self.width//2, self.height//2))
        self.grid.place_agent(hive, (self.width//2, self.height//2))


In [235]:
def agent_portrayal(agent):
    if isinstance(agent, Bee):
        if agent.dancing_baby:
             return {
            "color": "blue",
            "size": 50, }
            
        elif agent.following_dance:
             return {
            "color": "red",
            "size": 50, }
        else:
             return {
            "color": "orange",
            "size": 50, }
        
    elif isinstance(agent, Food):
         return {
                "color": "green",
                "size": 50, } 
    
    elif isinstance(agent, Hive):
         return {
                "shape": "triangle",
                "color": "yellow",
                "size": 500, }

In [236]:


model_params = {
    "N": {
        "type": "SliderInt",
        "value": 10,  
        "label": "Number of bees:",
        "min": 5,
        "max": 100,
        "step": 1,
    },
    "num_food": {
        "type": "SliderInt",
        "value": 10,  
        "label": "Number of food sources:",
        "min": 1,
        "max": 50,
        "step": 1,
    },
    "noise": {
        "type": "SliderInt",
        "value": 1,  
        "label": "Noise in bees' communication:",
        "min": 0,
        "max": 15,
        "step": 1,
    },
    "av_food_qual": {
        "type": "SliderInt",
        "value": 50,
        "label": "Average quality of food:",
        "min": 1,
        "max": 100,
        "step": 1,
    },
  "av_experiences": {
        "type": "SliderInt",
        "value": 50,
        "label": "Experience of bees:",
        "min": 0,
        "max": 100,
        "step": 1,
    },
     "quality_food_waggle": {
        "type": "SliderInt",
        "value": 50,
        "label": "Food quality of waggle dance:",
        "min": 0,
        "max": 100,
        "step": 1,
    },
     "quality_threshold": {
        "type": "SliderInt",
        "value": 50,
        "label": "Quality threshold:",
        "min": 0,
        "max": 100,
        "step": 1,
    },
     "memory_test": {
        "type": "SliderInt",
        "value": 50,
        "label": "Memory - Test mode:",
        "min": 0,
        "max": 100,
        "step": 1,
    },
     "mode_test": {
         "type": "Checkbox",
        "label": "Test mode",
         "value": False
    },
    "no_memories": {
        "type": "Checkbox",
        "label": "Restrict memory",
         "value": False
    },
   
   "width": 100,
   "height": 100
}

In [237]:
page = JupyterViz(
    BeeForagingModel,
    model_params,
    measures=["followers"],
    name="Waggle Dance",
    agent_portrayal=agent_portrayal,
)
page

Cannot show ipywidgets in text

-----------------------------------

-----------------------------------

----------------------------------

In [227]:
N = 100  
num_food = 
noise = 1  
av_food_qual = 50  
av_experiences = 50  
width = 100
height = 100
quality_food_waggle = 0
mode_test = False
quality_threshold = 50
memory_test = 0
no_memories = False

# This function run the model and give the percentage of bees that followed waggle dance

def bees_dancers_choose_conditions( N, num_food, width, height, noise, quality_threshold, av_food_qual, av_experiences, quality_food_waggle,
                                    no_memories, mode_test, memory_test):
                                   
    percentage = 0
    while percentage == 0:    
        model = BeeForagingModel( N, num_food, width, height, noise, quality_threshold, av_food_qual, av_experiences, quality_food_waggle,
                                  no_memories, mode_test, memory_test)

        num_steps = 2000

        for i in range(num_steps):
            model.step()

        dancers = sum(1 for bee in model.schedule.agents if isinstance(bee, Bee) and bee.following_dance)

        percentage =  ((dancers)  / model.num_agents)*100

    return percentage

In [228]:
bee_dancers = bees_dancers_choose_conditions(  N, num_food, width, height, noise, quality_threshold, av_food_qual, av_experiences, quality_food_waggle,
                 no_memories, mode_test, memory_test)

print("Percentage of bees that followed waggle dance:", bee_dancers)

Percentage of bees that followed waggle dance: 67.0


In [229]:
# Run the simulation 100 times
bee_dancers_percentages = []
for _ in range(100):
    percentage = bees_dancers_choose_conditions(N, num_food, width, height, noise, quality_threshold, av_food_qual, av_experiences, quality_food_waggle, no_memories, mode_test, memory_test)
    bee_dancers_percentages.append(percentage)

# Calculate the average percentage
average_percentage = np.mean(bee_dancers_percentages)

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(bee_dancers_percentages, label='Percentage of Bees Following Waggle Dance')
plt.hlines(average_percentage, 0, 100, colors='r', linestyles='dashed', label='Average Percentage')
plt.xlabel('Simulation Run')
plt.ylabel('Percentage')
plt.title('Percentage of Bees Following Waggle Dance Across 100 Simulations')
plt.legend()
plt.show()

print("Average percentage of bees that followed waggle dance:", average_percentage)

Average percentage of bees that followed waggle dance: 68.4


C:\Users\luisl\AppData\Local\Temp\ipykernel_10976\1207132564.py:18: UserWarning:

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.



-----------------

----------------

## Parameter Sweep

In [219]:
pip install plotly 

^C
Note: you may need to restart the kernel to use updated packages.


--------------------

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Ranges
experiences_range = np.arange(0, 85, 10)
qual_food_waggle_range = np.arange(30, 80, 10)
food_qual_range = np.arange(30, 80, 10)

temp_results = []

N = 100
num_food = 50
width = 100
height = 100
noise = 1
no_memories = False
quality_threshold = 50
mode_test = False
mem = 0

for exp in experiences_range:
    for food_waggle in qual_food_waggle_range:
        for food_quality in food_qual_range:
            dancers = bees_dancers_choose_conditions(N, num_food, width, height, noise, quality_threshold, food_quality, exp, food_waggle,
                                                     no_memories, mode_test, mem)

            temp_results.append([exp, food_waggle, food_quality, dancers])

results_1 = pd.DataFrame(temp_results, columns=['Experiences', 'AverageFoodQualityWaggle', 'AverageFoodQuality', 'PercentageFollowing'])

x = results_1['Experiences']
y = results_1['AverageFoodQualityWaggle']
z = results_1['AverageFoodQuality']
c = results_1['PercentageFollowing']

# Heatmap 3D
trace = go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(
        size=12,
        color=c,
        colorscale='Viridis',
        colorbar=dict(title='Percentage Following'),
        opacity=0.8
    )
)

layout = go.Layout(
    title='3D Heatmap of Bee Foraging Model Results',
    scene=dict(
        xaxis_title='Experiences',
        yaxis_title='Food Quality Waggle Dance',
        zaxis_title='Food Quality Around'
    ),
    margin=dict(l=0, r=0, b=0, t=0))

fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

experiences_range = np.arange(0, 75, 10)
qual_food_waggle_range = np.arange(25, 80, 10)
memory_range = np.arange(0, 70, 5)  
temp_results = []


N = 100
num_food = 50
width = 100
height = 100
noise = 1
no_memories = False
quality_threshold = 50
mode_test = True
food_quality = 50  


for exp in experiences_range:
    for food_waggle in qual_food_waggle_range:
        for mem in memory_range:
            dancers = bees_dancers_choose_conditions(
                N, num_food, width, height, noise, quality_threshold, 
                food_quality, exp, food_waggle, no_memories, mode_test, mem
            )
            temp_results.append([exp, food_waggle, food_quality, mem, dancers])

results_2 = pd.DataFrame(
    temp_results, 
    columns=['Experiences', 'AverageFoodQualityWaggle', 'FoodQuality', 'Memory', 'PercentageFollowing']
)

x = results_2['Experiences']
y = results_2['AverageFoodQualityWaggle']
z = results_2['Memory']
c = results_2['PercentageFollowing']

# Heatmap 3D
trace = go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(
        size=12,
        color=c,
        colorscale='Viridis',
        colorbar=dict(title='Percentage Following'),
        opacity=0.8
    )
)

layout = go.Layout(
    title='3D Heatmap of Bee Foraging Model Results',
    scene=dict(
        xaxis_title='Experiences',
        yaxis_title='Food Quality Waggle Dance',
        zaxis_title='Memory'
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()
